In [5]:
from pydub import AudioSegment
track = AudioSegment.from_file('audio/train_00003.m4a',  format='m4a')
track.set_channels(1)
track.set_frame_rate(16000)
file_handle = track.export('audio/train_00003.wav', format='wav')